In [488]:
%reset
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")
import torch
import numpy as np
import defmod as dm
import multimodule_usefulfunctions as mm 
import matplotlib.pyplot as plt
import defmod.hamiltonian_multishape as hamiltonian


torch.set_default_tensor_type(torch.DoubleTensor)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [489]:
nb_pts1 = 2
nb_pts2 = 3
nb_pts = [nb_pts1, nb_pts2]
dim = 2

o1 = torch.tensor([1.,1.])
o2 = torch.tensor([5.,5.])
origin = [o1,o2]
radius = [2.,1.]
numberPoints = [nb_pts1, nb_pts2]
gd_list = mm.multipleCircles(origin, radius, numberPoints)

In [490]:
# Defining the deformation modules
sigma1 = 2
manifold1 = dm.manifold.Landmarks(dim, nb_pts1, gd=gd_list[0].view(-1))
trans1 = dm.deformationmodules.Translations(manifold1, sigma1)

sigma2 = 2
manifold2 = dm.manifold.Landmarks(dim, nb_pts2, gd=gd_list[1].view(-1))
trans2 = dm.deformationmodules.Translations(manifold2, sigma2)

module_list = [trans1, trans2]
sigma_bg = 0.2
modules = dm.multishape.MultiShapeModule(module_list, sigma_bg)

In [491]:
# Setting cotangent variables
p1 = torch.cat([torch.ones(nb_pts1,1, requires_grad=True), torch.zeros(nb_pts1,1)],1).view(-1)
p2 = torch.cat([-torch.ones(nb_pts2,1,requires_grad=True), torch.zeros(nb_pts2,1)],1).view(-1)
cotan = [p1, p2, [p1, p2]]

modules.manifold.fill_cotan(cotan)

In [492]:
constr_Id = dm.constraints.Identity()
H = hamiltonian.Hamiltonian_multi(modules, constr_Id)

In [493]:
# 2 Modules
def controlslist2tensor(controls_list):
    return torch.cat([controls_list[0].view(-1),controls_list[1].view(-1), controls_list[-1][0].view(-1),controls_list[-1][1].view(-1)])

def controlstensor2list(controls, nb_pts, dim):
    n = sum(nb_pts)
    return [controls[:dim*nb_pts[0]], controls[dim*nb_pts[0]:dim*(nb_pts[0]+nb_pts[1])], [controls[dim*n:dim*(n+nb_pts[0])], controls[dim*(n+nb_pts[0]):]]]

def gdtensor2list(gdtensor, nb_pts, dim):
    gdlist = []
    gdlist_bg = []
    j = 0
    n = sum(nb_pts)
    for i in range(len(nb_pts)):
        gdlist.append(gdtensor[dim*j:dim*(j+nb_pts[i])])
        gdlist_bg.append(gdtensor[dim*(n+j):dim*(n+j+nb_pts[i])])
        j = j + nb_pts[i]
    return [*gdlist, gdlist_bg]

def gdlist2tensor(gdlist):
    return torch.cat([*gdlist[:-1],*gdlist[-1]],0)

In [420]:
def test_gradcheck_list2tensor():
    def list2tensor(controls):
        
        controls = controlstensor2list(controls, [nb_pts1, nb_pts2], dim)
        
        return controlslist2tensor(controls)
    
    controls = torch.cat([torch.ones(modules.controls[0].shape, requires_grad=True), torch.ones(modules.controls[0].shape, requires_grad=True)])

    return torch.autograd.gradcheck(list2tensor, controls, raise_exception=True)

test_gradcheck_list2tensor()

True

In [477]:
def test_gradcheck_man_fill():
    def man_fill(gd):
        
        out = dm.manifold.Landmarks(2, 10)
        out.fill(man, copy=True)
        
        return out.gd
    
    return torch.autograd.gradcheck(man_fill, (gd), raise_exception=True)


man = dm.manifold.Landmarks(2, 10)
man.fill_gd(torch.rand(2*10, requires_grad=True))

gd = man.gd

#test_gradcheck_man_fill()

## return false is normal

In [137]:
def test_gradcheck_cot_to_vs():
    def cot_to_vs(cotan):  
        
        man = mod.manifold.copy()
        
        if j in [-1, 2]:
            cotan = controlstensor2list(cotan, nb_pts, dim)

        man.fill_cotan(cotan) 

        field = man.cot_to_vs(1)
        return field(points)
    
    return torch.autograd.gradcheck(cot_to_vs, (cotan), raise_exception=True)

modules.compute_geodesic_variables(modules.manifold, constr_Id)

j = 2

mod = modules.module_list[j]
if j in [-1, 2]:
    cotan = controlslist2tensor(mod.manifold.cotan)
else:
    cotan = mod.manifold.cotan


points = torch.rand(dim*nb_pts1).view(-1,2)


test_gradcheck_cot_to_vs()

True

In [138]:
def test_gradcheck_innerprod():
    def innerprod(gd, mom):
        
        manifold.fill_gd(gd)
        manifold.fill_cotan(mom)  
                
        return mod.manifold.inner_prod_field(mod.field_generator())
    
    return torch.autograd.gradcheck(innerprod, (gd, mom), raise_exception=True)

manifold = dm.manifold.Landmarks(dim, nb_pts1)
mod = dm.deformationmodules.Translations(manifold, 1)

gd = torch.rand(nb_pts1*dim,1, requires_grad=True)
mom = torch.rand(nb_pts1*dim,1, requires_grad=True)

test_gradcheck_innerprod()

True

In [139]:
def test_gradcheck_autoaction():
    def autoaction(gd, mom):        
        
        H.module.manifold.fill_gd(gdtensor2list(gd, nb_pts, dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, nb_pts, dim))
        
        return H.module.autoaction()

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)


    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)

test_gradcheck_autoaction()

True

In [226]:
def test_gradcheck_geodesic_cont_self():
    def geodesic_cont_self(gd, mom):
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        H.module.compute_geodesic_control_from_self()

        return controlslist2tensor(H.module.controls)

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)

    return torch.autograd.gradcheck(geodesic_cont_self, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_cont_self()

True

In [241]:
def test_gradcheck_geodesic_controls():
    def geodesic_controls(gd, mom):
        modules.background.manifold.fill_gd([gd[:nb_pts1*dim], gd[nb_pts1*dim:]], copy=False)
        modules.background.manifold.fill_cotan([mom[:nb_pts1*dim], mom[nb_pts1*dim:]], copy=False)
        
        modules.background.compute_geodesic_control_from_self()

        return modules.background.controls
           

    gd = torch.cat(modules.background.manifold.gd.copy())
    gd.requires_grad_()
    mom = torch.cat(modules.background.manifold.cotan.copy())
    mom.requires_grad_()


    return torch.autograd.gradcheck(geodesic_controls, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_controls()

True

In [242]:
def test_gradcheck_geodesic_controls():
    def geodesic_controls(gd, mom):        
        man = H.module.manifold.copy()
        H.module.manifold.fill_gd(gdtensor2list(gd, nb_pts, dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, nb_pts, dim))
        
        H.module.compute_geodesic_control(man)

        return controlslist2tensor(H.module.controls)

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)
    

    return torch.autograd.gradcheck(geodesic_controls, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_controls()

True

In [506]:
def test_gradcheck_geodesic_var():
    def geodesic_var(gd, mom):
        
        H.module.manifold.fill_gd(gdtensor2list(gd, nb_pts, dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, nb_pts, dim))

        H.geodesic_controls()
        return controlslist2tensor(H.module.controls)
        #return H.module.l

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)

    return torch.autograd.gradcheck(geodesic_var, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_var()

RuntimeError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[ 5.9362e-02,  0.0000e+00,  5.9362e-02,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         -5.9362e-02,  0.0000e+00, -5.9362e-02,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-5.9362e-02,  0.0000e+00, -5.9362e-02,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          5.9362e-02,  0.0000e+00,  5.9362e-02,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000

In [185]:
def test_gradcheck_apply_mom():
    def apply_mom(gd, mom):

        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))

        modules.compute_geodesic_variables(modules.manifold, constr_Id)
        return H.apply_mom()

    
    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)


    return torch.autograd.gradcheck(apply_mom, (gd, mom), raise_exception=True)

test_gradcheck_apply_mom()

True

In [476]:
def test_gradcheck_Hamiltonian_2modules():
    def hamiltonian(gd, mom):
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        H.geodesic_controls()
        
        return H()
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)

#modules.compute_geodesic_variables(modules.manifold, constr_Id)

gd = gdlist2tensor(modules.manifold.gd)
mom = controlslist2tensor(modules.manifold.cotan)


test_gradcheck_Hamiltonian_2modules()

True

In [514]:
# check derivative of Hamiltonian wrt controls is zero

def Hamiltonian(controls):
    controls = controlstensor2list(controls, nb_pts, dim)
    modules.fill_controls(controls)
    H = hamiltonian.Hamiltonian_multi(modules, constr_Id)
    H.geodesic_controls()
    return H()

#controls = dm.usefulfunctions.flatten_tensor_list(modules.controls, [])
controls = controlslist2tensor(modules.controls)

a = torch.autograd.grad(Hamiltonian(controls), controls, retain_graph=True)

print(a)

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [486]:
def test_gradcheck_shooting():
    def shooting(gd, mom):
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        return dm.shooting.shoot_euler(H, it=2)
        #return dm.shooting.shoot_euler_multishape(H, it=2)
    return torch.autograd.gradcheck(shooting, (gd, mom), raise_exception=True,atol=1e-2)

gd = gdlist2tensor(modules.manifold.gd)
mom = controlslist2tensor(modules.manifold.cotan)


test_gradcheck_shooting()

True